Установка необходимых библиотек:

In [ ]:
! pip install transformers
! pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Загрузка предобработанных ранее данных:

In [ ]:
texts_preprocessed = []

with open(r'/content/texts_preprocessed.txt', 'r') as fp:  # текстовый файл заранее загружается в файлы
    for line in fp:
        x = line[:-1]
        texts_preprocessed.append(x)

In [ ]:
print(texts_preprocessed[0])

возможно доставаться слишком просто решать   опасение переключаться интерес знать   никуда деваться   становиться менее предсказуемый загадочный мужчина надолго пропадать 


Загрузка токенизатора и модели:

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru").cuda()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Сразу предобработать все 50 тысяч текстов не получится (даже с GPU). Поэтому я предобрабатывала по 1000 текстов, а затем записывала их эмбеддинги в список. Это повторялось 50 раз: то есть до тех пор, пока не векторизуется весь корпус.

In [ ]:
i = 0
k = 1000
answers_embeddings = []

while k <= 50000:
    encoded_input = tokenizer(texts_preprocessed[i:k], padding=True, truncation=True, max_length=24, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input.to('cuda'))
    answer_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    answers_embeddings.extend(answer_embeddings)
    i += 1000
    k += 1000

Нулевой элемент итогового списка выглядит так:

In [ ]:
answers_embeddings[0]

tensor([[ 0.2457, -0.1983, -0.2381,  ...,  0.7298, -0.1651, -0.1893],
        [-0.0682, -0.3474,  0.0652,  ..., -0.3770,  0.3249,  0.7633],
        [-0.3104, -0.3413, -0.4397,  ..., -0.2433,  0.0629,  0.1300],
        ...,
        [-0.3104, -0.3413, -0.4397,  ..., -0.2433,  0.0629,  0.1300],
        [ 1.2728,  0.1209,  0.3369,  ...,  0.8355, -0.1966,  0.3603],
        [ 0.6430,  0.0414,  0.6874,  ...,  0.3810, -0.6626, -0.0482]],
       device='cuda:0')

Элементы списка имеют следующий тип:

In [ ]:
type(answers_embeddings[0])

torch.Tensor

Список сохраняется на гугл диск, после чего загружается к себе на компьютер:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
path = '/content/gdrive/My Drive/answers_embeddings.pt'
torch.save(answers_embeddings, path)